In [1]:
import pandas as pd
import numpy as np
import csv
import os
from tqdm import tqdm

In [2]:
sim_save_dir = r'../data/simulations/market_scenarios_monte_carlo'
portfolio_run_out_dir = r'../data/simulations/portfolio_simulations_monte_carlo'


In [3]:
def gen_pf_wts(total_n, n, num_pf_wt_generations):
    final_list = []
    for i in range(num_pf_wt_generations):
        selected_positions = np.random.choice(range(total_n), size = n, replace=False)
        random_weights = np.random.rand(n)
        random_weights /= np.sum(random_weights)
        random_weights = np.round(random_weights,2)
        portfolio_weights = np.zeros(total_n)
        portfolio_weights[selected_positions] = random_weights
        final_list.append(portfolio_weights)

    return final_list

In [39]:
def write_results_to_csv(fname, year, pf_wts, portfolio_return, portfolio_vol):
    with open(os.path.join(portfolio_run_out_dir, fname+'_'+year+'.csv'), mode='a', newline='') as f:
        writer = csv.writer(f)    
        writer.writerow(np.append(np.append(pf_wts,portfolio_return), portfolio_vol))
    return True

In [4]:
def generate_portfolio_ret_vol(pf_wts, ret_np, vol_np, corr_np):
    # Assuming every input is np.array
    cov_matrix = np.outer(vol_np, vol_np) * corr_np
    
    portfolio_return = round(np.dot(pf_wts, ret_np),3)
    portfolio_vol = round(np.sqrt(np.dot(pf_wts.T, np.dot(cov_matrix, pf_wts))),3)

    return portfolio_return, portfolio_vol

In [ ]:
file_names = os.listdir(sim_save_dir)

In [13]:
file_names

['mc_sim_1000_year_2017_corr_noise_10.csv',
 'mc_sim_1000_year_2017_ret_noise_50.csv',
 'mc_sim_1000_year_2017_risk_noise_10.csv',
 'mc_sim_1001_year_2017_corr_noise_5.csv',
 'mc_sim_1001_year_2017_ret_noise_50.csv',
 'mc_sim_1001_year_2017_risk_noise_20.csv',
 'mc_sim_1002_year_2017_corr_noise_5.csv',
 'mc_sim_1002_year_2017_ret_noise_50.csv',
 'mc_sim_1002_year_2017_risk_noise_20.csv',
 'mc_sim_1003_year_2017_corr_noise_5.csv',
 'mc_sim_1003_year_2017_ret_noise_50.csv',
 'mc_sim_1003_year_2017_risk_noise_20.csv',
 'mc_sim_1004_year_2017_corr_noise_5.csv',
 'mc_sim_1004_year_2017_ret_noise_50.csv',
 'mc_sim_1004_year_2017_risk_noise_20.csv',
 'mc_sim_1005_year_2017_corr_noise_5.csv',
 'mc_sim_1005_year_2017_ret_noise_50.csv',
 'mc_sim_1005_year_2017_risk_noise_20.csv',
 'mc_sim_1006_year_2017_corr_noise_5.csv',
 'mc_sim_1006_year_2017_ret_noise_50.csv',
 'mc_sim_1006_year_2017_risk_noise_20.csv',
 'mc_sim_1007_year_2017_corr_noise_5.csv',
 'mc_sim_1007_year_2017_ret_noise_50.csv',
 'm

In [29]:
tt= file_names[0]
tt

'mc_sim_1000_year_2017_corr_noise_10.csv'

In [33]:
tt=tt.split('.')[:-1][0].split('_')


In [42]:
total_n = 25
k = 3
num_pf_wt_generations = 100000  # I want to restrict this number so that the final size of files is 
                                   # computationally tractable. Try num_gen = 1 lakh or 50K for n = 3
    
pf_wts_list = gen_pf_wts(total_n, k, num_pf_wt_generations)

for sim_num in tqdm(range(1,8400, 24)):
    filter_files = [x for x in file_names if 'mc_sim_'+str(sim_num)+'_' in x]    
    ret_fname = [x for x in filter_files if 'ret_noise' in x][0]
    risk_fname = [x for x in filter_files if 'risk_noise' in x][0]
    corr_fname = [x for x in filter_files if 'corr_noise' in x][0]

    annual_ret = pd.read_csv(sim_save_dir+'//'+ret_fname, header=None)
    annual_ret.columns = ['mf_id','annual_return']
    annual_vol = pd.read_csv(sim_save_dir+'//'+risk_fname, header=None)
    annual_vol.columns = ['mf_id','annual_vol']
    annual_corr = pd.read_csv(sim_save_dir+'//'+corr_fname, index_col=0)
    ret_np = np.array(annual_ret['annual_return'])
    vol_np = np.array(annual_vol['annual_vol'])
    corr_np = np.array(annual_corr)
    tt=ret_fname.split('.')[:-1][0].split('_')
    new_fname = 'mc_sim_' + str(tt[2])
    tyear = str(tt[4])
    
    for pf_wts in pf_wts_list:
        portfolio_return, portfolio_vol = generate_portfolio_ret_vol(pf_wts, ret_np, vol_np, corr_np)
        write_results_to_csv(new_fname, tyear,  pf_wts, portfolio_return, portfolio_vol)
        


100%|██████████████████████████████████████████████████████████████████████████████| 350/350 [2:07:13<00:00, 21.81s/it]
